# Orbit response

In [ ]:
import sys
import os
from os.path import join
from collections import Counter
from datetime import datetime
import h5py
from tqdm import tqdm
import numpy as np
from scipy import optimize as opt
from matplotlib import pyplot as plt
import proplot as pplt

sys.path.append('/Users/46h/Research/code/optimized_sav_gal/')
import sgfilter

pplt.rc['grid'] = False
pplt.rc['cmap.discrete'] = False
pplt.rc['cmap.sequential'] = 'dusk_r'

In [ ]:
def get_quad_id(filename):
    return filename.split('_')[5]

def get_quad_number(filename):
    return int(get_quad_id(filename)[2:])

def mean_std(x, f):
    N = np.sum(f)
    mean = np.sum(f * x) / N
    std = np.sum(f * (x - mean)**2) / N
    return mean, std

def autofilter(signal):
    n_opt = sgfilter.n_opt(signal)
    signal_filtered = sgfilter.sg_filter_gram(signal, n_opt, 2)
    return signal_filtered.values

In [ ]:
folder = 'Diagnostics/Data/Measurements/2022-04-22/'
filenames = os.listdir(folder)
filenames = [filename for filename in filenames if 'orbit_response' in filename]
filenames = sorted(filenames, key=get_quad_number, reverse=True)
filenames

In [ ]:
filename = filenames[0]
file = h5py.File(join(folder, filename), 'r')
list(file.keys())

In [ ]:
# Errors and warnings from log
for i in range(file['log'].size):
    if not(file['/log'][i, 'level'] == 'INFO'.encode('utf')):
        timestr = datetime.fromtimestamp(file['/log'][0, 'timestamp']).strftime("%m/%d/%Y, %H:%M:%S")
        print(f"{timestr} {file['log'][i, 'message']}")

# Configuration data
for key in file['/config'].keys():
    print(f"{key}")
    print("--------------")
    for name in file['/config'][key].dtype.names:
        print(f"{name}: {file['config'][key][name]}")
    print()

In [ ]:
data = file['scandata']
print('Data:')
for i, name in enumerate(data.dtype.names):
    print(f'{name} [{data.dtype[i]}]')
print()

slits = [key for key in data.dtype.names if 'PositionSync' in key]
print(f'slits: {slits}')

In [ ]:
for name in data.dtype.names:
    if name in ['timestamp', 'Cam09_profileX', 'Cam09_profileY']:
        continue
    fig, ax = pplt.subplots(figsize=(5.0, 2.0))
    ax.plot(data['iteration'], data[name], color='black')
    for i in [22, 44, 66, 88, 110]:
        ax.axvline(i, color='black', alpha=0.1)
    ax.format(xlabel='Iteration', ylabel=name)
    plt.show()

Some of the signals are just noise.

In [ ]:
profile = data['Cam09_profileX'][100]
profile = profile[75:]  # No signal at beginning.
noise_mean = np.mean(profile)

fig, ax = pplt.subplots(figsize=(4.0, 1.5))
ax.plot(profile, color='black', alpha=0.1, label='Signal')
ax.axhline(noise_mean, color='black', label='Noise mean')
ax.legend(loc='top')
plt.show()

Try thresholding to get rid of noise.

In [ ]:
thresh = 2e5
idx, profiles = [], []
for i, profile in enumerate(tqdm(data['Cam09_profileX'])):
    profile = profile[75:]
    profile -= noise_mean
    profile = np.clip(profile, thresh, None)
    profile -= thresh
    if np.count_nonzero(profile) > 60:
        idx.append(i)
        profiles.append(profile)
idx = np.array(idx)
profiles = np.array(profiles)

In [ ]:
fig, ax = pplt.subplots(figsize=(10, 5))
ax.pcolormesh(profiles.T)
plt.show()

In [ ]:
means = []
for i, profile in zip(idx, profiles):
    imax = np.argmax(profile)
    means.append(imax)

In [ ]:
means_long = np.full(len(data), np.nan)
for i, mean in zip(idx, means):
    means_long[i] = mean

fig, axes = pplt.subplots(nrows=2, figsize=(6, 2.25), height_ratios=[1.0, 0.4], spany=False)
plt_kws = dict(alpha=0.6)
axes[1].plot(data['HZ04_PositionSync'], **plt_kws)
axes[1].plot(data['HZ06_PositionSync'], **plt_kws)
axes[0].plot(means_long, color='black', lw=0, marker='.', ms=3)
axes[0].format(ylabel='Mean position', xlabel='Step')
plt.show()